In [1]:
import os
import openai
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import numpy as np

In [2]:
#Load openai key
openai.api_key = os.getenv("OPENAI_API_KEY")

## Create database for the first time

In [ ]:
#Load document and split it into chunks
loader = UnstructuredFileLoader('documents/codicedellastrada.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [9]:
#sanity check
print(texts[50].page_content)

15-bis. Salvo che il fatto costituisca reato, coloro che esercitano abusivamente, anche avvalendosi di altre persone, ovvero determinano altri ad esercitare abusivamente l’attività di parcheggiatore o guardiamacchine sono puniti con la sanzione amministrativa del pagamento di una somma da €729,00 a €2.918,00. Se nell’attività sono impiegati minori la somma è raddoppiata. Si applica, in ogni caso, la sanzione accessoria della confisca delle somme percepite, secondo le norme del capo I, sezione II, del titolo VI.

Articolo 8

Circolazione nelle piccole isole.


In [11]:
#create embedding db and store it in the knowledge/ folder
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embedding_function, persist_directory="knowledge/")
db.persist()

## Run queries

In [ ]:
#load db
embedding_function = OpenAIEmbeddings()
db=Chroma(persist_directory="knowledge/", embedding_function=embedding_function)

In [19]:
#instantiate qa object
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=db.as_retriever())

In [18]:
#test with a query in Italian
qa.run('Cosa si intende per strada a senso unico?')

' Una strada a senso unico è una strada su cui i veicoli possono viaggiare in una sola direzione.'

In [21]:
#test with a query in English
qa.run('What is the fine in case of speeding?')

' It depends on how much the speed limit is exceeded. If the speed limit is exceeded by up to 10 km/h, the fine is between €39 and €159. If the speed limit is exceeded by between 10 km/h and 40 km/h, the fine is between €159 and €639. If the speed limit is exceeded by between 40 km/h and 60 km/h, the fine is between €500 and €2,000. If the speed limit is exceeded by more than 60 km/h, the fine is between €779 and €3,119.'

In [9]:
#test with a query in English
qa.run('In which situations it is allowed to not wear a seatbelt?')

' According to Article 172 of the Italian driving code, it is allowed to not wear a seatbelt in the following situations: for members of the police force and municipal police carrying out an emergency service, for drivers and attendants of fire and ambulance services in case of emergency, for children under three years old travelling in vehicles with front airbags unless the airbag has been deactivated, and for children under 1.5 meters tall travelling in public service vehicles or vehicles for hire with a driver, provided they are accompanied by a passenger aged 16 or over.'

In [18]:
#test with a query in English
qa.run('What is the speed limit in urban areas?')

' 50 km/h'

## Embedding analysis

In [17]:
#explore the content of one element in the db
db.get('7a36234a-1cb2-11ee-9457-fb11816db9a1')

{'ids': ['7a36234a-1cb2-11ee-9457-fb11816db9a1'],
 'embeddings': None,
 'documents': ['8. Chiunque viola le disposizioni dei commi precedenti è soggetto alla sanzione amministrativa\n\ndel pagamento di una somma da €80,00 a €318,00.\n\n9.\n\nIl veicolo non può proseguire il viaggio se il conducente non abbia provveduto a sistemare il carico secondo le modalità stabilite dal presente articolo. Perciò l’organo accertatore, nel caso che trattasi di veicolo a motore, oltre all’applicazione della sanzione di cui al comma 8, procede al ritiro immediato della carta di circolazione e della patente di guida, provvedendo con tutte le cautele che il veicolo sia condotto in luogo idoneo per la detta sistemazione; del ritiro è fatta menzione nel verbale di contestazione della violazione. I documenti sono restituiti all’avente diritto allorché il carico sia stato sistemato in conformità delle presenti norme. Le modalità della restituzione sono fissate dal regolamento.\n\nArticolo 165\n\nTraino di ve

In [14]:
#in order to see the embeddings, add "include=['embeddings']"
#in this case we have 938 vectors of dimension 1536
embeddings=np.asarray(db.get(include=['embeddings'])['embeddings'])
embeddings.shape

(938, 1536)

In [26]:
#let's take a closer look at a couple of these embeddings
print(embeddings[10:12])

[[-0.01491894  0.00957309  0.00425001 ...  0.01969248 -0.02558462
  -0.00311191]
 [-0.01791039  0.00221909  0.00470427 ...  0.01587363 -0.01587363
   0.00166801]]


## Query analysis

In [13]:
#first step: embed the question and retrieve the most similar sentences in the db 
db.similarity_search('What is the speed limit on motorways?')

[Document(page_content='1. Ai fini della sicurezza della circolazione e della tutela della vita umana la velocità massima non può superare i 130 km/h per le autostrade, i 110 km/h per le strade extraurbane principali, i 90 km/h per le strade extraurbane secondarie e per le strade extraurbane locali, ed i 50 km/h per le strade nei centri abitati, con la possibilità di elevare tale limite fino ad un massimo di 70 km/h per le strade urbane le cui caratteristiche costruttive e funzionali lo consentano, previa installazione degli appositi segnali. Sulle autostrade a tre corsie più corsia di emergenza per ogni senso di marcia dotate di apparecchiature debitamente omologate per il calcolo della velocità media di percorrenza su tratti determinati, gli enti proprietari o concessionari possono elevare il limite massimo di velocità fino a 150 km/h sulla base delle caratteristiche progettuali ed effettive del tracciato, previa installazione degli appositi segnali, sempreché lo consentano l’intensi

In [34]:
#second step: call the API
#langchain completes this prompt by substituting {question} and {context} with the query and the sentences obtained through the similarity search respectively
qa.combine_documents_chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"